<a href="https://colab.research.google.com/github/pratikabhang/AI-Driven-Cybersecurity-Threat-Prediction-Platform/blob/main/SQL_TASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Write Python code to create a sample cybersecurity log table in SQLite, insert records, and define a helper function to run SQL queries and display results in Colab.


In [1]:
# STEP 1: Import libraries
import sqlite3
import pandas as pd

# STEP 2: Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# STEP 3: Create cyber_logs table
conn.execute("""
CREATE TABLE cyber_logs (
    id INTEGER PRIMARY KEY,
    user TEXT,
    ip_address TEXT,
    attack_type TEXT,
    country TEXT,
    src_bytes INT,
    dst_bytes INT,
    num_failed_logins INT,
    num_success_logins INT,
    port INT,
    timestamp TEXT,
    threat_text TEXT
);
""")

# STEP 4: Insert sample rows
sample_data = [
(1, 'alice', '192.168.1.10', 'DoS', 'USA', 2000, 500, 3, 1, 80, '2023-01-01 10:00:00', 'Suspicious login attempt'),
(2, 'bob', '192.168.1.11', 'Phishing', 'India', 3000, 700, 6, 0, 25, '2023-01-02 11:00:00', 'Phishing email detected'),
(3, 'eve', '192.168.1.12', 'PortScan', 'China', 1500, 200, 1, 2, 22, '2023-01-03 12:30:00', 'Multiple ports accessed'),
(4, 'alice', '192.168.1.10', 'DoS', 'USA', 2000, 500, 3, 1, 80, '2023-01-01 10:00:00', 'Duplicate log for testing'),
(5, 'charlie', '192.168.1.13', 'Malware', 'Germany', -500, 400, 2, 2, 443, '2023-01-04 09:15:00', 'Invalid negative bytes')
]

conn.executemany("""
INSERT INTO cyber_logs VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", sample_data)
conn.commit()

# STEP 5: Helper function to run SQL + return Pandas table
def run_query(query):
    return pd.read_sql(query, conn)


2. Write SQL query to remove duplicate cybersecurity log entries.

In [2]:

conn.execute("""
DELETE FROM cyber_logs
WHERE id NOT IN (
    SELECT MIN(id) FROM cyber_logs GROUP BY user, ip_address, attack_type, timestamp
)
""")
run_query("SELECT * FROM cyber_logs")


,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed
3,5,charlie,192.168.1.13,Malware,Germany,-500,400,2,2,443,2023-01-04 09:15:00,Invalid negative bytes


3. Write SQL query to replace missing source byte values with a default (e.g., 1000).

In [3]:
conn.execute("UPDATE cyber_logs SET src_bytes = 1000 WHERE src_bytes IS NULL")
run_query("SELECT * FROM cyber_logs")

,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed
3,5,charlie,192.168.1.13,Malware,Germany,-500,400,2,2,443,2023-01-04 09:15:00,Invalid negative bytes


4. Write SQL query to normalize IP addresses by trimming spaces.

In [4]:
conn.execute("UPDATE cyber_logs SET ip_address = TRIM(ip_address)")
run_query("SELECT DISTINCT ip_address FROM cyber_logs")


,ip_address
0,192.168.1.10
1,192.168.1.11
2,192.168.1.12
3,192.168.1.13


5. Write SQL query to filter out corrupted log records (negative byte counts).

In [5]:
conn.execute("DELETE FROM cyber_logs WHERE src_bytes < 0 OR dst_bytes < 0")
run_query("SELECT * FROM cyber_logs")


,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed


6. Write SQL query to standardize timestamp format in network logs.

In [6]:
run_query("SELECT id, strftime('%Y-%m-%d %H:%M:%S', timestamp) AS standardized_time FROM cyber_logs")


,id,standardized_time
0,1,2023-01-01 10:00:00
1,2,2023-01-02 11:00:00
2,3,2023-01-03 12:30:00


7. Write SQL query to clean threat intelligence text by removing unwanted characters (e.g., !).

In [7]:
conn.execute("UPDATE cyber_logs SET threat_text = replace(threat_text, '!', '')")
run_query("SELECT id, threat_text FROM cyber_logs")


,id,threat_text
0,1,Suspicious login attempt
1,2,Phishing email detected
2,3,Multiple ports accessed


8. Write SQL query to remove outlier records with failed login attempts greater than 10.

In [8]:
conn.execute("DELETE FROM cyber_logs WHERE num_failed_logins > 10")
run_query("SELECT * FROM cyber_logs")


,id,user,ip_address,attack_type,country,src_bytes,dst_bytes,num_failed_logins,num_success_logins,port,timestamp,threat_text
0,1,alice,192.168.1.10,DoS,USA,2000,500,3,1,80,2023-01-01 10:00:00,Suspicious login attempt
1,2,bob,192.168.1.11,Phishing,India,3000,700,6,0,25,2023-01-02 11:00:00,Phishing email detected
2,3,eve,192.168.1.12,PortScan,China,1500,200,1,2,22,2023-01-03 12:30:00,Multiple ports accessed


9. Write SQL query to count failed vs successful login attempts per user.

In [9]:
run_query("""
SELECT user, SUM(num_failed_logins) AS failed, SUM(num_success_logins) AS success
FROM cyber_logs GROUP BY user
""")


,user,failed,success
0,alice,3,1
1,bob,6,0
2,eve,1,2


10. Write SQL query to find the top 10 most frequent attack types.

In [10]:
run_query("""
SELECT attack_type, COUNT(*) AS frequency
FROM cyber_logs
GROUP BY attack_type
ORDER BY frequency DESC
LIMIT 10
""")


,attack_type,frequency
0,PortScan,1
1,Phishing,1
2,DoS,1


11. Write SQL query to calculate average source vs destination traffic (bytes).

In [11]:
run_query("SELECT AVG(src_bytes) AS avg_src, AVG(dst_bytes) AS avg_dst FROM cyber_logs")


,avg_src,avg_dst
0,2166.666667,466.666667


12. Write SQL query to calculate anomaly scores (z-score approximation) for failed logins.

In [12]:
run_query("""
SELECT user, num_failed_logins,
       (num_failed_logins - (SELECT AVG(num_failed_logins) FROM cyber_logs)) * 1.0 /
       (SELECT AVG(num_failed_logins) FROM cyber_logs) AS anomaly_score
FROM cyber_logs
""")


,user,num_failed_logins,anomaly_score
0,alice,3,-0.1
1,bob,6,0.8
2,eve,1,-0.7


13. Write SQL query to cluster IP addresses based on access frequency.

In [13]:
run_query("""
SELECT ip_address, COUNT(*) AS access_count
FROM cyber_logs GROUP BY ip_address ORDER BY access_count DESC
""")


,ip_address,access_count
0,192.168.1.12,1
1,192.168.1.11,1
2,192.168.1.10,1


14. Write SQL query to analyze port scanning attempts by counting frequency of ports.

In [14]:
run_query("""
SELECT port, COUNT(*) AS attempts
FROM cyber_logs GROUP BY port ORDER BY attempts DESC
""")


,port,attempts
0,80,1
1,25,1
2,22,1


15. Write SQL query to compute correlation between attack types and source countries.

In [15]:
run_query("""
SELECT attack_type, country, COUNT(*) AS occurrences
FROM cyber_logs GROUP BY attack_type, country ORDER BY occurrences DESC
""")


,attack_type,country,occurrences
0,DoS,USA,1
1,Phishing,India,1
2,PortScan,China,1


16. Write SQL query to predict intrusions based on failed login attempts.

In [16]:
run_query("""
SELECT id, user,
CASE WHEN num_failed_logins > 5 THEN 'Potential Intrusion' ELSE 'Normal' END AS prediction
FROM cyber_logs
""")


,id,user,prediction
0,1,alice,Normal
1,2,bob,Potential Intrusion
2,3,eve,Normal


17. Write SQL query to classify requests as malicious vs benign based on traffic thresholds.

In [17]:
run_query("""
SELECT id, user,
CASE WHEN src_bytes > 2500 OR dst_bytes > 500 THEN 'Malicious' ELSE 'Benign' END AS request_status
FROM cyber_logs
""")


,id,user,request_status
0,1,alice,Benign
1,2,bob,Malicious
2,3,eve,Benign


18. Write SQL query to forecast attack trends by counting attacks per day.

In [18]:
run_query("""
SELECT date(timestamp) AS attack_day, COUNT(*) AS attack_count
FROM cyber_logs GROUP BY attack_day ORDER BY attack_day
""")


,attack_day,attack_count
0,2023-01-01,1
1,2023-01-02,1
2,2023-01-03,1


19. Write SQL query to detect phishing attempts using keyword matching in threat text.

In [19]:
run_query("""
SELECT id, user,
CASE WHEN threat_text LIKE '%phish%' THEN 'Phishing Suspected' ELSE 'Normal' END AS phishing_flag
FROM cyber_logs
""")


,id,user,phishing_flag
0,1,alice,Normal
1,2,bob,Phishing Suspected
2,3,eve,Normal


20. Write SQL query to detect suspicious login attempts (failed > 3 and no successful logins).

In [20]:
run_query("""
SELECT id, user,
CASE WHEN num_failed_logins > 3 AND num_success_logins = 0 THEN 'Suspicious Login' ELSE 'Normal' END AS login_status
FROM cyber_logs
""")


,id,user,login_status
0,1,alice,Normal
1,2,bob,Suspicious Login
2,3,eve,Normal


21. Write SQL query to detect zero-day attacks (attack types that occur rarely).

In [21]:
run_query("""
SELECT id, attack_type,
CASE WHEN attack_type IN (
    SELECT attack_type FROM cyber_logs GROUP BY attack_type HAVING COUNT(*) < 2
) THEN 'Zero-Day Suspected' ELSE 'Known Attack' END AS anomaly_flag
FROM cyber_logs
""")


,id,attack_type,anomaly_flag
0,1,DoS,Zero-Day Suspected
1,2,Phishing,Zero-Day Suspected
2,3,PortScan,Zero-Day Suspected
